In [ ]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
OUTPUT_NAME = "system1"  # Change this to your desired output name
os.makedirs(f"../system/{OUTPUT_NAME}", exist_ok=True)

In [ ]:
!bin.bash
printf '16\\n0\\n' | gmx energy -f ../tmp/annealing_{OUTPUT_NAME}.edr -o ../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_temp.xvg
printf '22\\n0\\n' | gmx energy -f ../tmp/annealing_{OUTPUT_NAME}.edr -o ../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_z.xvg
printf '24\\n0\\n' | gmx energy -f ../tmp/annealing_{OUTPUT_NAME}.edr -o ../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_density.xvg´

In [ ]:
temp_file = f"../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_temp.xvg"
z_file = f"../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_z.xvg"
density_file = f"../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_density.xvg"

# Parse the XVG files
time_temp, temp_data = parse_xvg(temp_file)
time_z, z_data = parse_xvg(z_file)
time_density, density_data = parse_xvg(density_file)

# Print some basic stats
print(f"Temperature data points: {len(temp_data)}")
print(f"Z-coordinate data points: {len(z_data)}")
print(f"Density data points: {len(density_data)}")

In [ ]:
 Create a figure with a shared x-axis but multiple y-axes
fig = plt.figure(figsize=(12, 10))
gs = GridSpec(3, 1, height_ratios=[1, 1, 1], hspace=0.1)

# Create subplots
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)
ax3 = fig.add_subplot(gs[2], sharex=ax1)

# Plot temperature data
ax1.plot(time_temp, temp_data, 'r-', linewidth=1.5, label='Temperature')
ax1.set_ylabel('Temperature (K)', fontsize=12, color='red')
ax1.tick_params(axis='y', labelcolor='red')
ax1.grid(True, alpha=0.3)
ax1.set_title(f'MD Annealing Analysis for {OUTPUT_NAME}', fontsize=14)

# Plot z-coordinate data
ax2.plot(time_z, z_data, 'g-', linewidth=1.5, label='Z-coordinate')
ax2.set_ylabel('Z-coordinate (nm)', fontsize=12, color='green')
ax2.tick_params(axis='y', labelcolor='green')
ax2.grid(True, alpha=0.3)

# Plot density data
ax3.plot(time_density, density_data, 'b-', linewidth=1.5, label='Density')
ax3.set_ylabel('Density (kg/m³)', fontsize=12, color='blue')
ax3.set_xlabel('Time (ps)', fontsize=12)
ax3.tick_params(axis='y', labelcolor='blue')
ax3.grid(True, alpha=0.3)

# Hide x-labels for the top two plots
ax1.tick_params(axis='x', labelbottom=False)
ax2.tick_params(axis='x', labelbottom=False)

# Add a text annotation with some basic statistics
stats_text = f"""Temperature: Mean = {np.mean(temp_data):.2f} K, Min = {np.min(temp_data):.2f} K, Max = {np.max(temp_data):.2f} K
Z-coordinate: Mean = {np.mean(z_data):.2f} nm, Min = {np.min(z_data):.2f} nm, Max = {np.max(z_data):.2f} nm
Density: Mean = {np.mean(density_data):.2f} kg/m³, Min = {np.min(density_data):.2f} kg/m³, Max = {np.max(density_data):.2f} kg/m³"""

fig.text(0.5, 0.01, stats_text, ha='center', va='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

# Adjust layout
plt.tight_layout(rect=[0, 0.05, 1, 1])

# Save the figure
plt.savefig(f"../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_combined.png", dpi=300, bbox_inches='tight')

# Display the figure
plt.show()

In [ ]:
 Create a figure with a single subplot but three y-axes
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot temperature data on the primary y-axis
color1 = 'red'
ax1.set_xlabel('Time (ps)', fontsize=12)
ax1.set_ylabel('Temperature (K)', fontsize=12, color=color1)
ax1.plot(time_temp, temp_data, color=color1, linewidth=1.5, label='Temperature')
ax1.tick_params(axis='y', labelcolor=color1)

# Create a second y-axis for z-coordinate data
ax2 = ax1.twinx()
color2 = 'green'
ax2.set_ylabel('Z-coordinate (nm)', fontsize=12, color=color2)
ax2.plot(time_z, z_data, color=color2, linewidth=1.5, label='Z-coordinate')
ax2.tick_params(axis='y', labelcolor=color2)

# Create a third y-axis for density data (on the right side but positioned to the right of the second axis)
# We need to create a new axes instance with an invisible x-axis for this
ax3 = ax1.twinx()
# Offset the axis to make room for all three
ax3.spines['right'].set_position(('outward', 60))
color3 = 'blue'
ax3.set_ylabel('Density (kg/m³)', fontsize=12, color=color3)
ax3.plot(time_density, density_data, color=color3, linewidth=1.5, label='Density')
ax3.tick_params(axis='y', labelcolor=color3)

# Add a title
ax1.set_title(f'MD Annealing Analysis for {OUTPUT_NAME} (Combined View)', fontsize=14)

# Add grid
ax1.grid(True, alpha=0.3)

# Add a combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
ax3.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, loc='upper center', 
           bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=3)

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig(f"../system/{OUTPUT_NAME}/annealing_{OUTPUT_NAME}_combined_single.png", dpi=300, bbox_inches='tight')

# Display the figure
plt.show()